<a href="https://colab.research.google.com/github/KaterinaKinti/Deep-Learning/blob/main/TimeSeries_Adaboost_Xgboost_Stacking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ADABoost

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

#undersampler =RandomUnderSampler(sampling_strategy=0.5)#,'majority'
oversampler = SMOTE(sampling_strategy='minority') #, adasyn,sampling_strategy=0.1

AB = AdaBoostClassifier(n_estimators=50, learning_rate=1)

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)


for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    #scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Oversample the majority class
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)

    # Undersample the minority class
    #X_train_resampled, y_train_resampled = undersampler.fit_resample(X_train_resampled, y_train_resampled)

    #X_train_resampled, y_train_resampled = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
    #X_train_resampled, y_train_resampled = SMOTEENN(enn=EditedNearestNeighbours(sampling_strategy='majority'))


    AB.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)


    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
AB.fit(X_train_resampled, y_train_resampled)
y_pred = model.predict(X_test_scaled)
y_train_prob = model.predict_proba(X_train_resampled)[:, 1]
y_test_prob = model.predict_proba(X_test_scaled)[:, 1]
y_train_pred = model.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['AdaBoost'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve AdaBoost (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

XGBoost

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)

scaler = RobustScaler()

oversampler = SMOTE(sampling_strategy='minority')

XGB = XGBClassifier(class_weight='balanced', n_jobs=-1)

In [ ]:
# Same ratio of minority/majority in each fold
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=42)


for train_index, test_index in cv.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    #Scaling
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Oversampling
    X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train_scaled, y_train)


    XGB.fit(X_train_resampled, y_train_resampled)
    y_pred = model.predict(X_test_scaled)


    print(classification_report(y_test, y_pred))

In [ ]:
score = {}

In [ ]:
# Fit the model and make predictions
XGB.fit(X_train_resampled, y_train_resampled)
y_pred = XGB.predict(X_test_scaled)
y_train_prob = XGB.predict_proba(X_train_resampled)[:, 1]
y_test_prob = XGB.predict_proba(X_test_scaled)[:, 1]
y_train_pred = XGB.predict(X_train_resampled)


In [ ]:
# Calculate AUC score for both training and test data
train_auc = roc_auc_score(y_train_resampled, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# Print the AUC scores
print('Training AUC score: ', train_auc)
print('Test AUC score: ', test_auc)

score['XGBoost'] = [test_auc]

In [ ]:
# calculate ROC curve
train_fpr, train_tpr, train_thresholds = roc_curve(y_train_resampled, y_train_prob)
test_fpr, test_tpr, test_thresholds = roc_curve(y_test, y_test_prob)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
# Train Confusion Matrix
cma = confusion_matrix(y_train_resampled, y_train_pred)
print('Confusion matrix for Train data:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize=(10, 7))
sns.heatmap(df_cm, annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve XGBoost (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
Stacking

In [ ]:
#Stacking Classifier base:LR,RF,XGB  meta-learner: logistic regression
model_1 = LogisticRegression(C=3, class_weight='balanced', n_jobs=-1)
model_1.fit(train, y_train)
clf_1 = CalibratedClassifierCV(model_1, method='sigmoid')

model_2 = RandomForestClassifier(n_estimators=250, max_depth=25, class_weight='balanced', n_jobs=-1)
model_2.fit(train, y_train)
clf_2 = CalibratedClassifierCV(model_2, method='sigmoid')

model_3 = XGBClassifier(max_depth=5, n_estimators=100, class_weight='balanced', n_jobs=-1)
model_3.fit(train, y_train)
clf_3 = CalibratedClassifierCV(model_3, method='sigmoid')

stack_clf = StackingClassifier(classifiers=[clf_1, clf_2, clf_3], meta_classifier=LogisticRegression(), use_probas=True)
stack_clf.fit(train, y_train)

In [ ]:
# predict probabilities
y_probs_train = stack_clf.predict_proba(train)[:, 1]
y_probs_test = stack_clf.predict_proba(test)[:, 1]
y_predicted_train = stack_clf.predict(train)
y_predicted_test = stack_clf.predict(test)
train_auc = roc_auc_score(y_train, y_probs_train)
test_auc = roc_auc_score(y_test, y_probs_test)
train_acc = accuracy_score(y_train, y_predicted_train)
test_acc = accuracy_score(y_test, y_predicted_test)

In [ ]:
#Accuracy and AUC
print('*'*50)
print('Train AUC: %.3f' % train_auc)
print('Test AUC: %.3f' % test_auc)
print('*'*50)
print('Train Accuracy: %.3f' % train_acc)
print('Test Accuracy: %.3f' % test_acc)

# Plot ROC curve
train_fpr, train_tpr, _ = roc_curve(y_train, y_probs_train)
test_fpr, test_tpr, _ = roc_curve(y_test, y_probs_test)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.plot(train_fpr, train_tpr, marker='.', label='Train AUC')
plt.plot(test_fpr, test_tpr, marker='.', label='Test AUC')
plt.legend()
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()

In [ ]:
#Train Confusion Matrix
from sklearn.metrics import confusion_matrix

cma = confusion_matrix(y_train, y_predicted_train)
print('Confusion matrix:\n', cma)
df_cm = pd.DataFrame(cma, range(2), columns=range(2))
plt.figure(figsize = (10,7))
sns.heatmap(df_cm, annot=True,annot_kws={"size": 16}, fmt='g')

In [ ]:
#Precision-Recall AUC
precision, recall, thresholds = precision_recall_curve(y_test, y_test_prob)

fig = px.area(
    x=recall, y=precision,
    title=f'Precision-Recall Curve Stacking (AUC={auc(recall, precision):.4f})',
    labels=dict(x='Recall', y='Precision'),
    width=700, height=500
)
fig.add_shape(
    type='line', line=dict(dash='dash'),
    x0=0, x1=1, y0=1, y1=0
)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_xaxes(constrain='domain')

fig.show()

In [ ]:
from sklearn.metrics import precision_recall_curve, auc

# calculate precision-recall AUC scores for each model
pr_auc_scores = {}
for model_name, (y_pred_train, y_prob_train, y_pred_test, y_prob_test) in predictions.items():
    precision, recall, _ = precision_recall_curve(y_test, y_prob_test)
    pr_auc_scores[model_name] = auc(recall, precision)

# create pretty table for comparison of precision-recall AUC scores
pr_auc_table = PrettyTable(['Model', 'Precision-Recall AUC'])
for model_name, pr_auc_score in pr_auc_scores.items():
    pr_auc_table.add_row([model_name, pr_auc_score])

print('***************  Comparison of Precision-Recall AUC Scores  ****************')
print(pr_auc_table)
